[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/16Ty84XScfwuE4OIBTbzSTVZRnZ4ljWIo?usp=sharing)

## Carga de Librerias

In [ ]:
import os
import torch
from PIL import Image 
import matplotlib.pyplot as plt
import numpy as np
import gc
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets, models
import torchvision.utils
from collections import defaultdict
import torch.nn.functional as F

In [ ]:
from utils import *

### Verificación GPU

In [ ]:
if not torch.cuda.is_available():
  raise Exception("GPU not available. CPU training will be too slow.")

print("device name:", torch.cuda.get_device_name(0))

Exception: ignored

## Carga Datos

In [ ]:
pathTrain = ""
files = [os.path.splitext(f)[0] for f in os.listdir(pathTrain) if f.endswith('.jpg')]# + [f for f in os.listdir(path) if f.endswith('.png')]

## Carga Métodos 

In [ ]:
class SimDataset(Dataset):
  def __init__(self, phase, transform=None):
    if phase == 'train':
      self.input_images = np.array(ImageData(files[:40], pathTrain))
      self.target_masks = np.array(maskData(files[:40], pathMask))
    if phase == 'val':
      self.input_images = np.array(ImageData(files[40:50], pathTrain))
      self.target_masks = np.array(maskData(files[40:50], pathMask))
    if phase == 'test':
      self.input_images = np.array(ImageData(files[50:60], pathTrain))
      self.target_masks = np.array(maskData(files[50:60], pathMask))
    self.transform = transform

  def __len__(self):
    return len(self.input_images)

  def __getitem__(self, idx):
    image = self.input_images[idx]
    mask = self.target_masks[idx]
    if self.transform:
      image = self.transform(image)

    return [image, mask]

# use the same transformations for train/val in this example
trans = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]) # imagenet
])

train_set = SimDataset('train', transform = trans)
val_set = SimDataset('val', transform = trans)

batch_size = 5

dataloaders = {
  'train': DataLoader(train_set, batch_size=batch_size, shuffle=True, num_workers=0),
  'val': DataLoader(val_set, batch_size=batch_size, shuffle=True, num_workers=0)
}

NameError: ignored

## Carga Modelo y Pesos

In [ ]:
modelpath = ""
weights = "unet.hdf5"

if os.path.exists(modelpath):
    model = torch.load(modelpath)
    print("Model Loaded!!")
    lr = 1e-4

if os.path.exists(weights):
    model.load_weights(weights)
    print("Weights Loaded!!")
    lr = 1e-4

model.eval()

## Predicción

In [ ]:
# Get a batch of training data
for i_batch, sample_batched in enumerate(dataloaders['train']):
  inputs, masks = sample_batched
  print(i_batch, inputs.shape, masks.shape)
  plt.imshow(reverse_transform(inputs[2]))

In [ ]:
# Create a new simulation dataset for testing
test_dataset = SimDataset('test', transform = trans)
test_loader = DataLoader(test_dataset, batch_size=5, shuffle=False, num_workers=0)

# Get the first batch
inputs, labels = next(iter(test_loader))
inputs = inputs.to(device).float()
labels = labels.to(device)

# Predict
pred = model(inputs)
# The loss functions include the sigmoid function.
pred = torch.sigmoid(pred)
pred = pred.data.cpu().numpy()

# Change channel-order and make 3 channels for matplot
input_images_rgb = [x.transpose([1,2,0]) for x in inputs.cpu().numpy()]

# Map each channel (i.e. class) to each color
target_masks_rgb = [x.transpose([1,2,0]) for x in labels.cpu().numpy()]
pred_rgb = [x.transpose([1,2,0]) for x in pred]

In [ ]:
confidence = 0.90
plot_side_by_side([input_images_rgb, 
                   convert_mask(np.array(target_masks_rgb).transpose([0,3,1,2])),
                   convert_result(np.array(pred_rgb).transpose([0,3,1,2]),confidence)])